## 추천시스템 개발 코드

추천시스템은 고객별 최근 6개월의 데이터를 바탕으로 구축하였으며 6~3개월 전의 데이터를 1반기, 3개월~최근까지의 데이터를 2반기로 설정  
추천시스템은 2가지의 방식으로 구성되어 있으며 1번째 방식은 1반기에 많이 구매한 품목을 2반기에 구매하지 않은 경우 재구매를 추천하는것과  
2반기의 데이터를 바탕으로 knn최근접이웃방식과 잠재요인협업필터링의 방식을 통해 사용자,아이템사이의 유사도를 바탕으로 추천하는 방식 2가지를 사용  
이 중 2번째 방식에는 각 군집별로 가중치를 다르게 두어 맞춤형 추천시스템을 구현

### 1. 데이터 로딩 및 전처리

In [1]:
import pandas as pd
data = pd.read_csv('../../lmembers/data/purprod2.csv',encoding='ms949')
label_df = pd.read_excel('군집3(target_label=1).xlsx')
cate = pd.read_csv('../../lmembers/data/prodcl2.csv')
df = pd.merge(label_df,data)

In [2]:
df

,고객번호,라벨,제휴사,영수증번호,소분류코드,소분류명,통합분류,소비재분류,점포코드,구매일자,구매시간,구매금액,YEAR,월,분기,반기
0,1,0,A,2932858,A070102,국산냉장고.세탁기,대형가전,선매품,12,20141017,12,4014000,2014,10,Q4,H2
1,1,0,A,2932857,A070101,국산A/V,대형가전,선매품,12,20141017,12,1350000,2014,10,Q4,H2
2,1,0,A,3117914,A070101,국산A/V,대형가전,선매품,12,20141115,11,1350000,2014,11,Q4,H2
3,1,0,A,2888660,A090605,한실침구,인테리어,선매품,12,20141010,16,1380000,2014,10,Q4,H2
4,1,0,A,2868927,A090605,한실침구,인테리어,선매품,12,20141007,13,1380000,2014,10,Q4,H2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8084531,19346,2,C,8271720,C150101,국물봉지라면,식료품,편의품,265,20140814,18,500,2014,8,Q3,H2
8084532,19346,2,C,8271062,C030204,오이,식료품,편의품,265,20140521,21,660,2014,5,Q2,H1
8084533,19346,2,C,8271546,C060402,일반빵,식료품,편의품,265,20140720,21,660,2014,7,Q3,H2
8084534,19346,2,C,8271445,C030208,애호박,식료품,편의품,265,20140705,18,690,2014,7,Q3,H2


In [3]:

def load_pdf(df,cate,N):
    data3 = df.loc[(df.구매일자>=20151001)&(df.라벨 == N)]
    data6 = df.loc[(df.구매일자<20151001)&(df.구매일자>=20150601)&(df.라벨 == N)]
    data3_cate = pd.merge(data3,cate,on='소분류코드')
    data6_cate = pd.merge(data6,cate,on='소분류코드')
    # 상품분류명별 구매횟수를 기준으로 아이템 추천하기 위하여 피벗 생성
    pdf3_A = pd.pivot_table(data3_cate,
                        index = ['고객번호'],              # 행위치에 들어갈 열
                        columns = ['상품분류명'],         # 열위치에 들어갈 열
                        values = ['영수증번호'],              # 데이터로 사용할 열
                        aggfunc = ['count'])         # 데이터 집계 함수
    pdf6_A = pd.pivot_table(data6_cate,
                        index = ['고객번호'],           
                        columns = ['상품분류명'],       
                        values = ['영수증번호'],           
                        aggfunc = ['count'])    
    # 소분류명별 구매횟수를 기준으로 아이템 추천하기 위하여 피벗 생성
    pdf3_B = pd.pivot_table(data3_cate,
                        index = ['고객번호'],         
                        columns = ['소분류명_x'],       
                        values = ['영수증번호'],             
                        aggfunc = ['count'])        
    pdf6_B = pd.pivot_table(data6_cate,
                        index = ['고객번호'],           
                        columns = ['소분류명_x'],       
                        values = ['영수증번호'],           
                        aggfunc = ['count'])              
    # Nan값 0으로 대체 및 데이터프레임 정리
    pdf3_A.fillna(0,inplace=True)
    pdf3_A = pdf3_A['count']['영수증번호']
    pdf6_A.fillna(0,inplace=True)
    pdf6_A = pdf6_A['count']['영수증번호']
    pdf3_B.fillna(0,inplace=True)
    pdf3_B = pdf3_B['count']['영수증번호']
    pdf6_B.fillna(0,inplace=True)
    pdf6_B = pdf6_B['count']['영수증번호']
    return pdf3_A, pdf6_A, pdf3_B, pdf6_B

In [4]:
pdf3_0_중, pdf6_0_중, pdf3_0_소, pdf6_0_소 = load_pdf(df,cate,0)
pdf3_1_중, pdf6_1_중, pdf3_1_소, pdf6_1_소 = load_pdf(df,cate,1)
pdf3_2_중, pdf6_2_중, pdf3_2_소, pdf6_2_소 = load_pdf(df,cate,2)

### 2. 추천시스템 코드

**2.1 아이템 기반 최근접 이웃 협업 필터링**

In [5]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def predict_rating(ratings_arr, item_sim_arr):
    ratings_pred = ratings_arr.dot(item_sim_arr)/ np.array([np.abs(item_sim_arr).sum(axis=1)])
    return ratings_pred

def get_not_buy_list(ratings_matrix, custid):
    # custid로 입력받은 사용자의 모든 구매정보 추출하여 Series로 반환함. 

    # 반환된 user_rating 은 중분류명을 index로 가지는 Series 객체임. 
    user_rating = ratings_matrix.loc[custid,:]
    # user_rating이 0보다 크면 기존에 구매한 물품임. 대상 index를 추출하여 list 객체로 만듬
    already_buy = user_rating[ user_rating > 0].index.tolist()
    # 모든 중분류명을 list 객체로 만듬. 
    buy_list = ratings_matrix.columns.tolist()
    # 간편한 for문으로 already_buy에 해당하는 물품은 buy_list에서 제외함. 
    notbuy_list = [ item for item in buy_list if item not in already_buy]
    return notbuy_list

def recomm_item_by_custid(pred_df, custid, notbuy_list, top_n=10):
    # 구매예측 DataFrame에서 사용자id index와 notbuy_list로 들어온 중분류명 컬럼을 추출하여 가장 예측 구매횟수가 높은 순으로 정렬함. 
    recomm_items = pred_df.loc[custid, notbuy_list].sort_values(ascending=False)[:top_n]
    return recomm_items
    
def knn_recommend_pre(pdf3):
    # 코사인유사도는 행기준으로 작동하기에 transpose 함수로 전치
    pdf_T = pdf3.transpose()
    # 코사인 유사도 산출 (행인 아이템들을 기준으로 사용자간의 유사도)
    pdf_sim = cosine_similarity(pdf_T, pdf_T)
    # cosine_similarity()로 반환된 Numpy 행렬을 상품분류명으로 매핑해 DataFrame으로 변환
    pdf_sim_df = pd.DataFrame(data=pdf_sim, index=pdf3.columns,
                            columns=pdf3.columns)
    ratings_pred_knn = predict_rating(pdf3.values , pdf_sim_df.values)
    ratings_pred_matrix_knn = pd.DataFrame(data=ratings_pred_knn, index= pdf3.index,
                                    columns = pdf3.columns)
    
    return ratings_pred_matrix_knn

**2.2 잠재요인 협업 필터링**

In [6]:
import numpy as np
from sklearn.metrics import mean_squared_error

def get_rmse(R, P, Q, non_zeros):
    error = 0
    # 두개의 분해된 행렬 P와 Q.T의 내적 곱으로 예측 R 행렬 생성
    full_pred_matrix = np.dot(P,Q.T)
    
    # 실제 R 행렬에서 NULL이 아닌 값의 위치 인덱스 추출하여 실제 R 행렬과 예측 행렬의 RMSE 추출
    x_non_zero_ind = [non_zero[0] for non_zero in non_zeros]
    y_non_zero_ind = [non_zero[1] for non_zero in non_zeros]
    R_non_zeros = R[x_non_zero_ind, y_non_zero_ind]
    
    full_pred_matrix_non_zeros = full_pred_matrix[x_non_zero_ind,y_non_zero_ind]
    
    mse = mean_squared_error(R_non_zeros, full_pred_matrix_non_zeros)
    rmse = np.sqrt(mse)
    
    return rmse

# 행렬 분해
def matrix_factorization(R, K, steps=200, learning_rate=0.01, r_lambda=0.01):
    num_users, num_items = R.shape
    np.random.seed(1)
    P = np.random.normal(scale=1./K, size = (num_users,K))
    Q = np.random.normal(scale=1./K, size = (num_items,K))
    
    break_count = 0
    
    # R > 0인 행 위치, 열 위치, 값을 non_zeros 리스트 객체에 저장
    non_zeros = [(i,j,R[i,j]) for i in range(num_users) for j in range(num_items) if R[i,j] > 0]
    
    # SGD 기법으로 P와 Q 매트릭스를 계속 업데이트
    for step in range(steps):  # steps는 SGD의 반복횟수
        for i, j, r in non_zeros:
            # 실제 값과 예측 값의 차이인 오류 값 구함
            eij = r - np.dot(P[i,:],Q[j,:].T)
            
            P[i,:] = P[i,:] + learning_rate*(eij * Q[j,:] - r_lambda*P[i,:])
            Q[j,:] = Q[j,:] + learning_rate*(eij * P[i,:] - r_lambda*Q[j,:])
            
        rmse = get_rmse(R,P,Q, non_zeros)
        if ( step % 10) == 0: # 10회 반복할 때마다 오류 값 출력
            print(f'iteration step: {step}, rmse: {rmse}')
    return P, Q
    
def SGD_fac_recommend_pre(pdf3,steps=200,K=50,learning_rate=0.001,r_lambda = 0.001):
    P, Q = matrix_factorization(pdf3.values, steps=steps, K=K,learning_rate= learning_rate, r_lambda = r_lambda)
    pred_matrix = np.dot(P, Q.T)
    ratings_pred_matrix = pd.DataFrame(data=pred_matrix, index= pdf3.index,
                                   columns = pdf3.columns)
    return ratings_pred_matrix

In [7]:
# 군집별 예측구매횟수 매트릭스 csv파일로 저장
def pred_matrix_save(df,cate,N):
    pdf3_중, _, pdf3_소, _ = load_pdf(df,cate,N)
    knn_중 = knn_recommend_pre(pdf3_중)
    knn_소 = knn_recommend_pre(pdf3_소)
    sgd_중 = SGD_fac_recommend_pre(pdf3_중)
    sgd_소 = SGD_fac_recommend_pre(pdf3_소)
    knn_중.to_csv(f'./dataset/knn_{N}_중.csv')
    knn_소.to_csv(f'./dataset/knn_{N}_소.csv')
    sgd_중.to_csv(f'./dataset/sgd_{N}_중.csv')
    sgd_소.to_csv(f'./dataset/sgd_{N}_소.csv')

In [8]:
# pred_matrix_save(df,cate,0)
# pred_matrix_save(df,cate,1)
# pred_matrix_save(df,cate,2)

In [9]:
# 1반기에 많이 구입한 물품 중 2반기 구입하지 않은 물품을 추천
def recom_notbuy(custid,pdf3,pdf6):
    user_rating = pdf6.loc[custid,:]
    already_buy = user_rating[ user_rating > 0].sort_values(ascending=False)
    notbuy_list = get_not_buy_list(pdf3, custid)
    for x in already_buy.index:
        if x in notbuy_list:
            already_buy.drop(index=x ,axis=0,inplace=True)
    recom_list = already_buy
    return recom_list

In [10]:
# 중분류명 기준으로 추천
def get_recom_list_item_중(custid,pdf3_중,pdf6_중,knn_중,sgd_중,top):
    notbuy_list = get_not_buy_list(pdf3_중, custid)
    recomm_items_knn_중 = recomm_item_by_custid(knn_중, custid, notbuy_list, top_n=top)
    recomm_items_knn_중 = pd.DataFrame(data=recomm_items_knn_중.values,index=recomm_items_knn_중.index,columns=['pred_score'])
    recomm_items_sgd_중 = recomm_item_by_custid(sgd_중, custid, notbuy_list, top_n=top)
    recomm_items_sgd_중 = pd.DataFrame(data=recomm_items_sgd_중.values,index=recomm_items_sgd_중.index,columns=['pred_score'])
    recomm_items_knn_중.index.name='상품분류명'
    recomm_items_sgd_중.index.name='상품분류명'
    recom_items_중 = pd.merge(recomm_items_sgd_중,recomm_items_knn_중,on='상품분류명')
    recom_items_중['recom_pred'] = recom_items_중.iloc[:,1]*recom_items_중.iloc[:,0]
    recom_items_중 = recom_items_중.sort_values(by='recom_pred',ascending=False)
    recom_list_중 = recom_notbuy(custid,pdf3_중,pdf6_중)
    return recom_items_중,recom_list_중

In [11]:
# 추천받은 중분류명 기준으로 소분류명을 추천
def get_recom_list_item_소(custid,recom_items_중,recom_list_중,pdf6_소,c,M,N):
    data3 = df.loc[(df.구매일자>=20151001)&(df.라벨 == c)]
    data6 = df.loc[(df.구매일자<20151001)&(df.구매일자>=20150601)&(df.라벨 == c)]
    data3_cate = pd.merge(data3,cate,on='소분류코드')
    data6_cate = pd.merge(data6,cate,on='소분류코드')
    knn_소 = pd.read_csv(f'./dataset/knn_{c}_소.csv',index_col=0)
    sgd_소 = pd.read_csv(f'./dataset/sgd_{c}_소.csv',index_col=0)
    recom_소_list = {}
    recom_소_items = {}
    for i in recom_list_중[:M].index:
        recom_소 = data6_cate.소분류명_x[data6_cate.상품분류명 == i].unique().tolist()
        recom_소_list[i] = pdf6_소.loc[custid,recom_소].sort_values(ascending=False)[:N].index.tolist()
    for i in recom_items_중[:M].index:
        recom_소 = data3_cate.소분류명_x[data3_cate.상품분류명 == i].unique().tolist()
        recomm_items_knn_소 = knn_소.loc[custid,recom_소]
        recomm_items_sgd_소 = sgd_소.loc[custid,recom_소]
        recomm_items_knn_소.index.name='상품분류명'
        recomm_items_sgd_소.index.name='상품분류명'
        recom_items_소 = pd.merge(recomm_items_knn_소,recomm_items_sgd_소,on='상품분류명')
        recom_items_소['recom_pred'] = recom_items_소.iloc[:,1]*recom_items_소.iloc[:,0]
        recom_items_소 = recom_items_소.sort_values(by='recom_pred',ascending=False)
        recom_소_items[i] = recom_items_소.index[:N].tolist()
    return recom_소_items,recom_소_list

### 3. 각 감소고객군집별로 가중치 부여

In [12]:
cate = pd.read_csv('../../lmembers/data/prodcl2.csv')
data_cate = pd.merge(data,cate,on='소분류코드')

In [13]:
list_A = data_cate.상품분류명[data_cate.제휴사_x == 'A'].unique().tolist()
list_B = data_cate.상품분류명[data_cate.제휴사_x == 'B'].unique().tolist()
list_C = data_cate.상품분류명[data_cate.제휴사_x == 'C'].unique().tolist()
list_편의 = data_cate.상품분류명[data_cate.소비재분류_x == '편의품'].unique().tolist()
list_선매 = data_cate.상품분류명[data_cate.소비재분류_x == '선매품'].unique().tolist()
list_전문 = data_cate.상품분류명[data_cate.소비재분류_x == '전문품'].unique().tolist()
list_레저 = data_cate.상품분류명[data_cate.통합분류_x == '레저/취미'].unique().tolist()
list_패션 = data_cate.상품분류명[data_cate.통합분류_x == '패션의류/잡화'].unique().tolist()
list_명품 = data_cate.상품분류명[data_cate.통합분류_x == '명품/쥬얼리'].unique().tolist()
list_식품 = data_cate.상품분류명[data_cate.통합분류_x == '식료품'].unique().tolist()
list_기호 = data_cate.상품분류명[data_cate.통합분류_x == '기호품'].unique().tolist()
list_생활 = data_cate.상품분류명[data_cate.통합분류_x == '생활잡화'].unique().tolist()

## 1번군집 통합 추천 시스템

In [14]:
# 1번군집 통합 추천 시스템
# 1번군집은 A사의 고급제품을 중심으로한 마케팅이 필요하기 때문에 A사와 전문품, 선매품과 매출이 많이 감소한 패션,명품, 레저에 가중치를 주었다.
def recommend_list_1(custid,df,cate,c,top = 50,M=3,N=3):
    pdf3_중, pdf6_중, _, pdf6_소 = load_pdf(df,cate,c)
    sgd_중 = pd.read_csv(f'./dataset/sgd_{c}_중.csv',index_col=0)
    knn_중 = pd.read_csv(f'./dataset/knn_{c}_중.csv',index_col=0)
    for i in knn_중.columns:
        if i in list_A:
            knn_중[i] = knn_중[i] *1.5
        if i in list_전문:
            knn_중[i] = knn_중[i] *1.5
        if i in list_선매:
            knn_중[i] = knn_중[i] *1.2
        if i in list_레저:
            knn_중[i] = knn_중[i] *1.2
        if i in list_패션:
            knn_중[i] = knn_중[i] *1.2
        if i in list_명품:
            knn_중[i] = knn_중[i] *1.5
    recom_items_중, recom_list_중 = get_recom_list_item_중(custid,pdf3_중,pdf6_중,knn_중,sgd_중,top)
    recom_소_items,recom_소_list = get_recom_list_item_소(custid,recom_items_중,recom_list_중,pdf6_소,c,M,N)
    print(f'3개월전에 {recom_소_list}물품들을 구매하셨는데 지금은 필요하지 않으신가요?')
    print()
    print(f'{custid}고객님이 구매하신 상품을 구매한 고객들이 많이 구매한 {recom_소_items}은/는 어떠세요?')

In [15]:
cust_0 = np.random.choice(df.고객번호[df.라벨 == 0].unique().tolist())
recommend_list_1(cust_0,df,cate,0,M=3,N=3)

3개월전에 {'샐러드/간편채소': ['샐러드', '간편구색', '드레싱'], '양식': ['서양델리', '식당가 양식', '양식'], '가공식품': ['전문베이커리', '떡', '한과(세트행사)']}물품들을 구매하셨는데 지금은 필요하지 않으신가요?

14650고객님이 구매하신 상품을 구매한 고객들이 많이 구매한 {'농산물': ['채소', '농산가공', '농산단기행사'], '명품': ['명품잡화', 'global 편집샵', '어덜트'], '레스토랑': ['브랜드샵', '식당가 한식', '포숑']}은/는 어떠세요?


## 2번군집 통합 추천 시스템

In [16]:
# 2번군집 통합 추천 시스템
# 2번군집은 A,B사의 편의품분류와 식료품, 기호품을 중심으로 마케팅이 필요하기에 이에 가중치를 부여
def recommend_list_2(custid,df,cate,c,top = 50,M=3,N=3):
    pdf3_중, pdf6_중, _, pdf6_소 = load_pdf(df,cate,c)
    sgd_중 = pd.read_csv(f'./dataset/sgd_{c}_중.csv',index_col=0)
    knn_중 = pd.read_csv(f'./dataset/knn_{c}_중.csv',index_col=0)
    for i in knn_중.columns:
        if i in list_A:
            knn_중[i] = knn_중[i] *1.5
        if i in list_B:
            knn_중[i] = knn_중[i] *1.5
        if i in list_편의:
            knn_중[i] = knn_중[i] *1.5
        if i in list_식품:
            knn_중[i] = knn_중[i] *1.5
        if i in list_기호:
            knn_중[i] = knn_중[i] *1.5
        if i in list_선매:
            knn_중[i] = knn_중[i] *1.2  
        if i in list_패션:
            knn_중[i] = knn_중[i] *1.2  
        
    recom_items_중, recom_list_중 = get_recom_list_item_중(custid,pdf3_중,pdf6_중,knn_중,sgd_중,top)
    recom_소_items, recom_소_list = get_recom_list_item_소(custid,recom_items_중,recom_list_중,pdf6_소,c,M,N)
    print(f'3개월전에 {recom_소_list}물품들을 구매하셨는데 지금은 필요하지 않으신가요?')
    print()
    print(f'{custid}고객님이 구매하신 상품을 구매한 고객들이 많이 구매한 {recom_소_items}은/는 어떠세요?')

In [17]:
cust_1 = np.random.choice(df.고객번호[df.라벨 == 1].unique().tolist())
recommend_list_2(cust_1,df,cate,1,M=3,N=3)

3개월전에 {'농산물': ['청과', '농산가공', '채소'], '축산가공': ['유제품', '햄', '수입산육포'], '수산품': ['즉석구이김류', '생선', '냉동식품']}물품들을 구매하셨는데 지금은 필요하지 않으신가요?

51고객님이 구매하신 상품을 구매한 고객들이 많이 구매한 {'가공식품': ['일반가공식품', '전문베이커리', '수입단기행사'], '젓갈/반찬': ['벌크김치', '장류', '가공행사'], '소고기': ['우육', '한우안심', '한우설도']}은/는 어떠세요?


## 3번군집 통합 추천 시스템

In [35]:
# 3번군집 통합 추천 시스템
# 3번군집은 1,2번군집보다 구매가 고르게 분산되어 있으므로 가중치도 분산되게 부여 
# C사의 비중이 군집중 가장높고 편의,전문품 분류의 감소와 기호품,식료품,명품,생활잡화의 감소가 높으므로 가중치를 부여
def recommend_list_3(custid,df,cate,c,top = 50,M=3,N=3):
    pdf3_중, pdf6_중, _, pdf6_소 = load_pdf(df,cate,c)
    sgd_중 = pd.read_csv(f'./dataset/sgd_{c}_중.csv',index_col=0)
    knn_중 = pd.read_csv(f'./dataset/knn_{c}_중.csv',index_col=0)
    for i in knn_중.columns:
        if i in list_C:
            knn_중[i] = knn_중[i] *1.5
        if i in list_편의:
            knn_중[i] = knn_중[i] *1.3
        if i in list_전문:
            knn_중[i] = knn_중[i] *1.3
        if i in list_기호:
            knn_중[i] = knn_중[i] *1.3
        if i in list_식품:
            knn_중[i] = knn_중[i] *1.3
        if i in list_명품:
            knn_중[i] = knn_중[i] *1.3
        if i in list_생활:
            knn_중[i] = knn_중[i] *1.3
    recom_items_중, recom_list_중 = get_recom_list_item_중(custid,pdf3_중,pdf6_중,knn_중,sgd_중,top)
    recom_소_items, recom_소_list = get_recom_list_item_소(custid,recom_items_중,recom_list_중,pdf6_소,c,M,N)
    print(f'3개월전에 {recom_소_list}물품들을 구매하셨는데 지금은 필요하지 않으신가요?')
    print()
    print(f'{custid}고객님이 구매하신 상품을 구매한 고객들이 많이 구매한 {recom_소_items}은/는 어떠세요?')

In [36]:
cust_2 = np.random.choice(df.고객번호[df.라벨 == 2].unique().tolist())
recommend_list_3(3828,df,cate,2,M=5,N=5)

3개월전에 {'주방용품': ['수입주방', '수 저', '국산주방', '뚝배기', '식기건조대/수저통'], '식기': ['수입도자기', '국산도자기', '그라스', '전통도자기', '크리스탈'], '생활잡화': ['식기단기행사', '매트', '롤티슈', '각티슈/미용티슈', '칼/가위'], '캐주얼': ['영 컨템포러리', '캐릭터캐주얼', '영 캐릭터', 'SPACE 5.1', 'Taste5.1'], '내의': ['패션내의', '란제리', '팬티', '남성팬티', '남성런닝셔츠']}물품들을 구매하셨는데 지금은 필요하지 않으신가요?

3828고객님이 구매하신 상품을 구매한 고객들이 많이 구매한 {'농산물': ['청과', '유기농채소', '채소', '농산가공', '건과'], '명품': ['수입의류', '컨템포러리', '명품잡화', '수입잡화', '디자이너'], '아동': ['수입 아동복', 'N/B 아동복', 'L/C 아동복', '토들러', '유아복'], '수산품': ['생선', '건생선', '김류', '냉동식품', '건어물류'], '주류': ['음료', '주류', '공병', '한차']}은/는 어떠세요?


https://www.figma.com/file/afJteiISjEHAy5r65mODHX/prototype?node-id=0%3A1